#### TFT API 에 대해서 설명되어있는 사이트 : https://velog.io/@twa02189/TFT-API-%EB%8D%B0%EC%9D%B4%ED%84%B0-%EC%84%A4%EB%AA%85

In [1]:
import pandas as pd
import requests
import pprint
import json
import time
from pandas import json_normalize

pd.set_option('display.max_colwidth', None) ## 셀 최대 너비 제한 해제

#### 기본 키 저장 + baser URL 저장 + request header 저장

In [2]:
# api_key 저장
api_key = 'RGAPI-39d1115b-559c-49b0-bacb-cd3ea0588e94' ## 만료되면 갱신해줘야함 거의 하루마다

# request header 저장
request_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key 
    }

# base_url 저장
base_url = "https://kr.api.riotgames.com/tft/" ## 이 뒤에다가 + 로 그때그떄 필요한 URL 을 더해줄 것

#### 마스터, 그랜드 마스터 구간 유저들의 닉네임 추출 

In [3]:
## 현재 그마와 마스터 티어밖에 사람이 존재하지 않으므로 마스터 티어의 데이터만 조회

# grand master

grandmaster_url = base_url + 'league/v1/grandmaster'

grandmaster = requests.get(grandmaster_url, headers = request_header).json()

# master

master_url = base_url + 'league/v1/master'

master = requests.get(master_url, headers = request_header).json()

In [4]:
# 가져온 마스터 데이터 프레임 조회 (마스터 랭크에 속한 유저 조회)
pd.DataFrame(master).head(3)

# 가져온 그랜드마스터 데이터 프레임 조회 (그랜드마스터 랭크에 속한 유저 조회)
pd.DataFrame(grandmaster).head(3)

,tier,leagueId,queue,name,entries
0,GRANDMASTER,f1e46b7f-5703-3dad-b9c5-6a8f5dfa5598,RANKED_TFT,Soraka's Enforcers,"{'summonerId': 'VVpwkqbuN3VVwG378diAEHea6jFrq1ADYBQAEstZizthf7k', 'summonerName': '이렇게 좋은 날', 'leaguePoints': 348, 'rank': 'I', 'wins': 88, 'losses': 45, 'veteran': False, 'inactive': False, 'freshBlood': True, 'hotStreak': True}"
1,GRANDMASTER,f1e46b7f-5703-3dad-b9c5-6a8f5dfa5598,RANKED_TFT,Soraka's Enforcers,"{'summonerId': 'KKtx2l6oWWPWxU6Pi5tm0trZQI6yFkVbHjyMALGori8cG24DSiXPdy5gug', 'summonerName': '천연만화', 'leaguePoints': 418, 'rank': 'I', 'wins': 74, 'losses': 32, 'veteran': False, 'inactive': False, 'freshBlood': True, 'hotStreak': False}"
2,GRANDMASTER,f1e46b7f-5703-3dad-b9c5-6a8f5dfa5598,RANKED_TFT,Soraka's Enforcers,"{'summonerId': 'BhJKOIvlHJRa_Ra0uNNDlrqN46i6snagNkH_p8VZR28GJA', 'summonerName': 'Teukhan', 'leaguePoints': 321, 'rank': 'I', 'wins': 79, 'losses': 51, 'veteran': False, 'inactive': False, 'freshBlood': True, 'hotStreak': False}"


In [5]:
# master_df 에 저장
master_df = pd.DataFrame(master)

# grandmaster_df 에 저장
grandmaster_df = pd.DataFrame(grandmaster)

##### 결측치 제거 및 json normalize 테스트

In [6]:
# entries 컬럼 확장 
master_entries_normalized = json_normalize(master_df['entries'])

# 확장된 데이터프레임 병합
master_df = pd.concat([master_df.drop(['entries'], axis=1), master_entries_normalized], axis=1)

# 필요없는 컬럼 drop('queue', 'name')
master_df = master_df.drop(['leagueId', 'queue', 'name', 'rank'], axis=1) ## 이 네가지는 당연하고 필요없는 정보

# summonerName 컬럼이 결측치인 행 제거
master_df = master_df.dropna(subset=['summonerName'])

# 확인
master_df ## null 값이 아니라 공란 그자체가 값으로 존재하는 듯함

# summonerName 컬럼이 공란인 행 제거
master_df = master_df[master_df['summonerName'] != '']

# 확인
master_df.head() # 결측치 제거된 것 확인했음

,tier,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,MASTER,CwFvZnFIBnI5FwTXLx4lvp4zHrSa8EuE958nh8q-C3csDxs,쁘보배,116,77,45,False,False,True,False
1,MASTER,dtilHdI9usYxEelmpdrCfyTywsYSL_Odbl0g3e3DUvn99eY,싸커우,37,52,30,False,False,True,False
2,MASTER,J61nNdwOutomkBA23KtetozIhFIpA9R3x2nkuJp3iKy3uBRvKpJU9bfsLA,쇼린고,31,93,76,False,False,True,False
3,MASTER,drrMcPUMslSdtEJ4S-joF7McUIwwjaqbi3jkI8ws4L2denVKES2ug3QaYQ,PS 카나타,228,70,32,False,False,True,False
4,MASTER,2tcDWfWuiYh5NwzIWVToPwUVyhfygQPKmLi8G-AnJuj9E14,ZENIA TFT,19,60,31,False,False,True,False


##### 결측치 제거 및 json 펼치는 함수 생성

In [7]:
def json_normalize_and_dropna(df):
    # entries 컬럼 확장
    master_entries_normalized = json_normalize(df['entries'])
    
    # 확장된 데이터프레임 병합
    df = pd.concat([df.drop(['entries'], axis=1), master_entries_normalized], axis=1)
    
    # 필요없는 컬럼 drop('queue', 'name')
    df = df.drop(['leagueId', 'queue', 'name', 'rank'], axis=1)
    
    # summonerName 컬럼이 결측치인 행 제거
    df = df.dropna(subset=['summonerName'])
    
    # summonerName 컬럼이 공란인 행 제거
    df = df[df['summonerName'] != '']
    
    return df

In [8]:
# grandmaster df 결측치 제거 및 json normalize
grandmaster_df = json_normalize_and_dropna(grandmaster_df)

#### 추출한 데이터 프레임에서 닉네임을 모두 뽑아서 리스트로 작성

##### 닉네임 하나 골라서 puuid 추출 테스트

In [9]:
## 한명의 닉네임을 넣어서 제대로 나오는지 확인

# 닉네임을 넣어줄 url 저장
id_url = base_url + "summoner/v1/summoners/by-name/COPY O"

# 닉네임을 넣었을 때, 반환값 저장
random_id = requests.get(id_url, headers = request_header).json()

# puuid 추출
random_id['puuid'] ## puuid 추출 확인 / random_id 는 딕셔너리 형태(json)

't_Wf-ad5Drr5FzTmxmkRnsEtBNUf-7PRoTll2k8kxN23sYTKJs2VOrOjeQ418PF31x2QQ2X02RrAtw'

##### 앞서서 만든 master_df 에서 닉네임만 리스트로 추출 (df 에서 닉네임만 리스트로 추출하는 함수 생성)

In [10]:
# 닉네임 리스트 추출 함수(닉네임 공란도 제거)
def get_summoner_names_list(df):
    df = df[df['summonerName'] != '']
    summoner_names_list = df['summonerName'].tolist()
    return summoner_names_list

# master_df 의 닉네임 리스트 추출
master_summoner_names_list = get_summoner_names_list(master_df)

# grandmaster_df 의 닉네임 리스트 추출
grandmaster_summoner_names_list = get_summoner_names_list(grandmaster_df)

# 확인
print(master_summoner_names_list[:3])

print(grandmaster_summoner_names_list[:3])

['쁘보배', '싸커우', '쇼린고']
['이렇게 좋은 날', '천연만화', 'Teukhan']


##### 닉네임의 리스트를 넣으면 puuid 의 리스트로 반환하는 함수 생성

In [11]:
# 마스터 puuid 리스트 추출 함수
def get_puuid_list(nickname_list):
    puuid_list = []
    
    for nickname in nickname_list:
        # 닉네임을 넣어줄 url 저장
        id_url = base_url + f"summoner/v1/summoners/by-name/{nickname}"
        
        # 닉네임을 넣었을 때, 반환값 저장
        random_id = requests.get(id_url, headers=request_header).json()
        
        # puuid 추출하여 리스트에 추가
        if 'puuid' in random_id:
           puuid_list.append(random_id['puuid'])
    
    return puuid_list

# 마스터 puuid 리스트 추출해서 master_puuid_list 에 저장
master_puuid_list = get_puuid_list(master_summoner_names_list)

# 그랜드 puuid 리스트 추출
grandmaster_puuid_list = get_puuid_list(grandmaster_summoner_names_list)

['eVDGTA0QwoU8cXfzJj7jCsMkmlWKUyqnuwvo1Se-1O02ibxeaJPH-Qu2vl9LkRTwROkqJx7TS_PNKA', 'PDEcFJx7ZoWmThh882YMZsU6lfuQ51jYuFqVssWjsdtyD1EbrorT_MCEe852xPiw-FQRE5NIlxTq3Q', 'gpD9IlxyZMowxr43BRxKmVQE1q_my3OVTuW9B3FHxUCfXGUjy0i4bQeGPqsFdVsgY9wcDZR-YqL9oQ', '0B2Ddpu4J4Up4ZwTeQ_7R6_wMfVg-TCQELKds6QHUmfhoBuMPgVFq97lXH75j0gHLqLOiFEoS7-M2g', 'kph2AvVxfMcfYgeWyFXscvdzCinl4_e_AO6TcmVDNv25WQjNKkPnDF5_G58MTpHn8wrI3uHmhNN3Bg']


In [15]:
# 확인
print(master_puuid_list[:3])

print(grandmaster_puuid_list[:3])

['eVDGTA0QwoU8cXfzJj7jCsMkmlWKUyqnuwvo1Se-1O02ibxeaJPH-Qu2vl9LkRTwROkqJx7TS_PNKA', 'PDEcFJx7ZoWmThh882YMZsU6lfuQ51jYuFqVssWjsdtyD1EbrorT_MCEe852xPiw-FQRE5NIlxTq3Q', 'gpD9IlxyZMowxr43BRxKmVQE1q_my3OVTuW9B3FHxUCfXGUjy0i4bQeGPqsFdVsgY9wcDZR-YqL9oQ']
['rx1nmyxEddDyQujL2nvjfiHV_LDcBEo-zyf16l3BFnXQExOMoB6tNss-WGlc_GRgcac2tg1GhDgBhg', 'lskGGafPGWafTOrBtnSVRCyYdggWRybYvU_B0r65qQvnGCU2C4gmZmt9rMDAnruQuoloPyWnpeBiqQ', 'Qd68F2UMKe7FMb-qpbUs3njG6k_4ZLwr2FRG0zHruSDr9XTtZOZ1cdAXyLd_kGAt0_IheL5QMQLdvQ']
